# Notes

First iteration is to model: goal to have end to end pipeline from data to dynamics for a patient
- Don't have test train split
- Predict on training data
- Entry is an exercise
- Include exercise meta like https://pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/ 

In [1]:
import os 
import re
import csv
import json
from datetime import datetime

import math
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import classification_report

# Constants

In [2]:
SEQ_MAX_LEN = 600
NUM_CLASSES = 6
NUM_EXERCISES = 9
NUM_FLAG_BS = 2

In [3]:
dir_data_root = os.path.join('..', 'data')
dir_exercises = os.path.join(dir_data_root, 'json', 'exercises_raw')
dir_exercises_augmented = os.path.join(dir_data_root, 'json', 'exercises_augmented')
dir_patiens_sessions = os.path.join(dir_data_root, 'json', 'patients_sessions')

# Data prep

### Build training set

In [4]:
regions = [
    'frontal',
    'orbital',
    'oral'
]

In [5]:
regions = {
    '0_LefteyeMidbottom': 'orbital', 
    '1_LefteyeMidtop': 'orbital',  
    '2_LefteyeInnercorner': 'orbital', 
    '3_LefteyeOutercorner': 'orbital',  
    '4_LefteyebrowInner': 'frontal', 
    '5_LefteyebrowCenter': 'frontal',  
    '6_RighteyeMidbottom': 'orbital',  
    '7_RighteyeMidtop': 'orbital', 
    '8_RighteyeInnercorner': 'orbital',  
    '9_RighteyeOutercorner': 'orbital', 
    '10_RighteyebrowInner': 'frontal', 
    '11_RighteyebrowCenter': 'frontal',  
    '12_NoseTip': 'frontal', 
    '13_MouthLowerlipMidbottom': 'oral',
    '14_MouthLeftcorner': 'oral',
    '15_MouthRightcorner': 'oral',
    '16_MouthUpperlipMidtop': 'oral',
    '17_ChinCenter': 'oral', 
    '18_ForeheadCenter': 'frontal', 
    '19_LeftcheekCenter': 'oral', 
    '20_RightcheekCenter': 'oral',
}

In [6]:
def build_region(pois, region):
    exercise_sequence_region = []
    
    for poi in sorted(pois.keys()):
        if regions[poi] != region: continue
        
        sequences = pois[poi]
        exercise_sequence_region.append(sequences['xs'])
        exercise_sequence_region.append(sequences['ys'])
        exercise_sequence_region.append(sequences['zs'])

    exercise_sequence_region = pad_sequences(
        exercise_sequence_region,
        padding="pre",
        maxlen=SEQ_MAX_LEN)
        
    return exercise_sequence_region

In [7]:
def exercise_to_input(file_path):
    with open(file_path, 'r') as f_r:
        exercise = json.load(f_r)
        
        # build global
        exercise_sequence_global_region = []

        for poi in sorted(exercise['pois'].keys()):
            sequences = exercise['pois'][poi]
            exercise_sequence_global_region.append(sequences['xs'])
            exercise_sequence_global_region.append(sequences['ys'])
            exercise_sequence_global_region.append(sequences['zs'])
    
        exercise_sequence_global_region = pad_sequences(
            exercise_sequence_global_region,
            padding="pre",
            maxlen=SEQ_MAX_LEN)
        
        # build regions 
        exercise_sequence_frontal_region = build_region(exercise['pois'], 'frontal')
        exercise_sequence_oral_region = build_region(exercise['pois'], 'oral')
        exercise_sequence_orbital_region = build_region(exercise['pois'], 'orbital')
        
    # build meta
    x_a_1 = [0] * NUM_EXERCISES
    x_a_2 = [0] * NUM_FLAG_BS
    x_a_1[exercise['meta']['id']] = 1
    x_a_2[exercise['meta']['flag_before_surgery']] = 1

    return [
            x_a_1 + x_a_2, 
            exercise_sequence_global_region, 
            exercise_sequence_frontal_region,
            exercise_sequence_oral_region,
            exercise_sequence_orbital_region,
            exercise['meta']['evaluation']
           ]

In [8]:
xslist_meta = list()
xslist_global = list()
xslist_frontal = list()
xslist_oral = list()
xslist_orbital = list()
yslist = list()

exercises_sources = [
    dir_exercises,
    dir_exercises_augmented
]

for exercise_source in exercises_sources:
    for file_name in os.listdir(exercise_source):
        file_path = os.path.join(exercise_source, file_name)

        if file_name == '.DS_Store': continue

        _xs_meta, _xs_global, _xs_frontal, _xs_oral, _xs_orbital, _ys = exercise_to_input(file_path)

        yslist.append(_ys)
        xslist_meta.append(_xs_meta)  
        xslist_global.append(_xs_global)
        xslist_frontal.append(_xs_frontal)
        xslist_oral.append(_xs_oral)
        xslist_orbital.append(_xs_orbital)
        #break
            
ys = np.array(yslist)
xs_meta = np.array(xslist_meta)   
xs_global = np.array(xslist_global) 
xs_frontal = np.array(xslist_frontal) 
xs_oral = np.array(xslist_oral) 
xs_orbital = np.array(xslist_orbital) 

print(ys.shape)
print(xs_meta.shape)
print(xs_global.shape)
print(xs_frontal.shape)
print(xs_oral.shape)
print(xs_orbital.shape)

(10740,)
(10740, 11)
(10740, 63, 600)
(10740, 18, 600)
(10740, 21, 600)
(10740, 24, 600)


## Modeling

In [9]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [10]:
def get_dnn(inputLayer): 
    m = Dense(4, activation="relu")(inputLayer)
    m = Model(inputs=inputLayer, outputs=m)

    return m

In [11]:
def get_cnn(inputLayer):
    chanDim = -1
    
    m = Conv1D(16, 3, padding='same', activation='relu')(inputLayer)
    m = BatchNormalization(axis=chanDim)(m)
    m = MaxPooling1D((2))(m)
    m = Conv1D(32, 3, padding='same', activation='relu')(m)
    m = BatchNormalization(axis=chanDim)(m)
    m = MaxPooling1D((2))(m)
    m = Conv1D(64, 3, padding='same', activation='relu')(m)
    m = BatchNormalization(axis=chanDim)(m)
    m = MaxPooling1D((2))(m)
    m = Conv1D(64, 3, padding='same', activation='relu')(m)
    m = BatchNormalization(axis=chanDim)(m)
    m = MaxPooling1D((2))(m)
    m = Flatten()(m)
    m = Dropout(0.5)(m)
    m = Dense(128, activation="relu")(m)
    m = Model(inputs=inputLayer, outputs=m)

    return m

In [12]:
def get_model():
    input_meta = Input(shape=xs_meta.shape[1:])
    model_meta = get_dnn(input_meta)
    
    input_global = Input(shape=xs_global.shape[1:])
    model_global = get_cnn(input_global)
    
    input_frontal = Input(shape=xs_frontal.shape[1:])
    model_frontal = get_cnn(input_frontal)  

    input_oral = Input(shape=xs_oral.shape[1:])
    model_oral = get_cnn(input_oral)  
    
    input_orbital = Input(shape=xs_orbital.shape[1:])
    model_orbital = get_cnn(input_orbital)  
    
    
    model_contatenate = concatenate([
        model_meta.output, 
        model_global.output,
        model_frontal.output,
        model_oral.output,
        model_orbital.output,
    ])
    
    model_contatenate = Dense(32, activation="relu")(model_contatenate)
    model_contatenate = Dense(6, activation="softmax")(model_contatenate)
    model = Model(inputs=[
        model_meta.input,
        model_global.input,
        model_frontal.input,
        model_oral.input,
        model_orbital.input
    ], outputs=model_contatenate)

    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer=Adam(learning_rate=1e-3, decay=1e-3 / 200),
        metrics=['accuracy']
    )
    
    
    return model

In [13]:
test_model = get_model()
test_model.summary()

Metal device set to: Apple M1


2023-06-17 08:45:56.627369: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-17 08:45:56.627525: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 63, 600)]    0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 18, 600)]    0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 21, 600)]    0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 24, 600)]    0           []                               
                                                                                            

 batch_normalization_10 (BatchN  (None, 5, 64)       256         ['conv1d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_14 (BatchN  (None, 6, 64)       256         ['conv1d_14[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 max_pooling1d_2 (MaxPooling1D)  (None, 7, 64)       0           ['batch_normalization_2[0][0]']  
                                                                                                  
 max_pooling1d_6 (MaxPooling1D)  (None, 2, 64)       0           ['batch_normalization_6[0][0]']  
                                                                                                  
 max_pooli

                                                                                                  
Total params: 265,046
Trainable params: 263,638
Non-trainable params: 1,408
__________________________________________________________________________________________________


In [14]:
print(set(ys))

{0, 1, 2, 3, 4, 5}


In [15]:
weight_training_classes = {0: 0.32075471698113206, 
                           1: 1.1333333333333333,
                           2: 1.3076923076923077, 
                           3: 3.4, 
                           4: 3.4,
                           5: 1.5454545454545454}

### K-fold evaluation

In [16]:
k = 5
train = 0.8
val = 0.2
test = 0.2

In [17]:
VERBOSE = 0
EPOCHS = 200
BATCH_SIZE = 8

In [18]:
from sklearn.model_selection import KFold

In [19]:
def get_k_indx(k, n):

    k_fold = KFold(n_splits=k)
    train_ = []
    val_ = []
    test_ = []
    indx = []

    for train_indices, test_indices in k_fold.split(ys):
        n_k = len(train_indices)
        val_split = int(n_k * train)
        indx.append([train_indices[:val_split],train_indices[val_split + 1:], test_indices])
    
    return indx

In [20]:
indxs = get_k_indx(k, len(ys))

for i in range(k):
    train_indx, val_indx, test_indx  = indxs[i]
    xs_meta_i = xs_meta[train_indx]
    xs_meta_i_val = xs_meta[val_indx]
    xs_meta_i_test = xs_meta[test_indx]
    
    xs_global_i = xs_global[train_indx]
    xs_global_i_val = xs_global[val_indx]
    xs_global_i_test = xs_global[test_indx]
    
    xs_frontal_i = xs_frontal[train_indx]
    xs_frontal_i_val = xs_frontal[val_indx]
    xs_frontal_i_test = xs_frontal[test_indx]
    
    xs_oral_i = xs_oral[train_indx]
    xs_oral_i_val = xs_oral[val_indx]
    xs_oral_i_test = xs_oral[test_indx]
    
    xs_orbital_i = xs_orbital[train_indx]
    xs_orbital_i_val = xs_orbital[val_indx]
    xs_orbital_i_test = xs_orbital[test_indx]
    
    ys_i = ys[train_indx]
    ys_i_val = ys[val_indx]
    ys_i_test = ys[test_indx]

    model = get_model()

    model.fit(
        x=[
            xs_meta_i, 
            xs_global_i, 
            xs_frontal_i,
            xs_oral_i,
            xs_orbital_i], y=ys_i, 
        validation_data=([
            xs_meta_i_val,
            xs_global_i_val,
            xs_frontal_i_val,
            xs_oral_i_val,
            xs_orbital_i_val], ys_i_val),
        batch_size=BATCH_SIZE, epochs=EPOCHS,
        class_weight=weight_training_classes,
        verbose=VERBOSE)
    
    
    
    y_pred = model.predict([
        xs_meta_i_test,
        xs_global_i_test,
        xs_frontal_i_test,
        xs_oral_i_test,
        xs_orbital_i_test],verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)

    print(classification_report(ys_i_test, y_pred_bool))

2023-06-17 08:45:58.982472: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-17 08:46:01.050177: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 08:46:47.124846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 11:12:39.410840: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       1.00      0.94      0.97       964
           1       0.88      0.99      0.93       347
           2       0.96      0.99      0.98       352
           3       0.99      1.00      1.00       124
           4       0.98      1.00      0.99       126
           5       0.99      1.00      0.99       235

    accuracy                           0.97      2148
   macro avg       0.97      0.99      0.98      2148
weighted avg       0.97      0.97      0.97      2148



2023-06-17 11:12:46.724598: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 11:13:35.227741: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 13:37:17.974657: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.99      0.96      0.97       967
           1       0.96      0.98      0.97       337
           2       0.96      0.99      0.97       347
           3       0.91      1.00      0.95       128
           4       0.97      0.98      0.98       129
           5       0.97      0.99      0.98       240

    accuracy                           0.97      2148
   macro avg       0.96      0.98      0.97      2148
weighted avg       0.97      0.97      0.97      2148



2023-06-17 13:37:26.400763: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 13:38:18.291677: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 16:06:09.524879: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.99      0.92      0.95       961
           1       0.94      0.98      0.96       357
           2       0.93      0.99      0.96       357
           3       0.86      0.99      0.92       120
           4       0.94      0.99      0.96       123
           5       0.96      0.96      0.96       230

    accuracy                           0.95      2148
   macro avg       0.94      0.97      0.95      2148
weighted avg       0.96      0.95      0.95      2148



2023-06-17 16:06:17.304279: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 16:07:10.677020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 18:34:46.708187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.99      0.97      0.98       960
           1       0.99      0.95      0.97       356
           2       0.88      0.99      0.93       358
           3       0.99      0.93      0.96       120
           4       0.94      0.96      0.95       124
           5       1.00      0.97      0.98       230

    accuracy                           0.97      2148
   macro avg       0.96      0.96      0.96      2148
weighted avg       0.97      0.97      0.97      2148



2023-06-17 18:34:56.215052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 18:35:54.561866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 21:04:03.977866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.98      0.97      0.98       968
           1       0.99      0.97      0.98       338
           2       0.93      0.99      0.96       346
           3       0.98      0.98      0.98       128
           4       0.97      0.96      0.96       128
           5       1.00      0.97      0.98       240

    accuracy                           0.97      2148
   macro avg       0.97      0.97      0.97      2148
weighted avg       0.98      0.97      0.97      2148



In [21]:
Stop

NameError: name 'Stop' is not defined

In [22]:
import statistics
statistics.mean([0.98, 0.97, 0.95, 0.96, 0.97])

0.966

# Patient dynamics

In [ ]:
from operator import attrgetter

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('_mpl-gallery')

In [ ]:
weights = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0]
exercises_number = 9

## Single patient with details

In [ ]:
patient_id = '00000000020'

In [ ]:
sessions = []
with open(os.path.join(dir_patiens_sessions, '%s.json' % patient_id), 'r') as f_r:
    sessions = json.load(f_r)
    
print(sessions)

In [ ]:
sessions.sort(key=lambda x: x['exercise_dates'])
print(sessions)

In [ ]:
dynamic = []
evaluations = []

for session in sessions:
    exercises_result = []
    print(session['evaluation'] + 1) #from index to class
    evaluations.append(session['evaluation'] + 1)

    fig, axs = plt.subplots(1,9,figsize=(15,2)) 

    for exercise_id in range(exercises_number):
        file_name = os.path.join(
            dir_exercises, '%s_%s_%s.json' % (
                patient_id, 
                session['id'],
                exercise_id))
        
        if not os.path.isfile(file_name):
            continue
            
        xs_a, xs_b, ys = exercise_to_input(file_name)
        
        xs_a = np.array([xs_a])   
        xs_b = np.array([xs_b]) 
        
        #print(xs_a.shape)
        #print(xs_b.shape)
        
        
        y_pred = model.predict([xs_a, xs_b], verbose=0)
        exercises_result.append(list(y_pred[0]))
        
                
        axs[exercise_id].bar(1 + np.arange(6), y_pred[0], width=1, edgecolor="white", linewidth=0.7)

        axs[exercise_id].set(
            xlim=(0, 7), 
            xticks=np.arange(1, 7),
            ylim=(0, 1))
    
    #average session 
    prediction = np.average(np.array(exercises_result), axis=0)
    exercise_score = [y*w for y, w in zip(prediction,weights)]
    dynamic.append(sum(exercise_score))
    
    fig, axs = plt.subplots(1,2,figsize=(4,2)) 
        
    x = 1 + np.arange(6)
    y_pred = y_pred[0]
    y_score = exercise_score

    axs[0].bar(x, y_pred, width=1, edgecolor="white", linewidth=0.7)

    axs[0].set(xlim=(0, 7), 
              xticks=np.arange(1, 7),
              ylim=(0, 1), 
             )


    axs[1].bar(x, y_score, width=1, edgecolor="white", linewidth=0.7)
    axs[1].set(xlim=(0, 7), 
              xticks=np.arange(1, 7),
              ylim=(0, max(y_score) + 1), 
             )

    plt.show()    
        

print(dynamic)
fig, ax = plt.subplots(figsize=(4,2)) 

x = 1 + np.arange(len(sessions))
y = dynamic

#ax.bar(x, y, width=1, edgecolor="white", linewidth=0.7)
ax.plot(x, y, color='tab:blue')
ax.bar(x, evaluations, color='tab:orange', edgecolor="white", linewidth=0.7)

ax.set(xlim=(0, len(sessions) + 1), 
          xticks=np.arange(1, len(sessions) + 1),
          ylim=(0, max(7, max(y) + 1)), 
         )

plt.show() 

## Patients Dynamics: Multiple Patients

In [ ]:
def get_fine_score(patient_id):
    sessions = []
    fine_score = []
    evaluations = []

    with open(os.path.join(dir_patiens_sessions, '%s.json' % patient_id), 'r') as f_r:
        sessions = json.load(f_r)
        
    sessions.sort(key=lambda x: x['exercise_dates'])
    
    for session in sessions:
        exercises_result = []
        evaluations.append(session['evaluation'] + 1)
    
        for exercise_id in range(exercises_number):
            file_name = os.path.join(
                dir_exercises, '%s_%s_%s.json' % (
                    patient_id, 
                    session['id'],
                    exercise_id))

            if not os.path.isfile(file_name):
                continue

            xs_a, xs_b, ys = exercise_to_input(file_name)

            xs_a = np.array([xs_a])   
            xs_b = np.array([xs_b]) 

            y_pred = model.predict([xs_a, xs_b], verbose=0)
            exercises_result.append(list(y_pred[0]))

        #average session 
        prediction = np.average(np.array(exercises_result), axis=0)
        exercise_score = [y*w for y, w in zip(prediction,weights)]
        fine_score.append(sum(exercise_score))
        
    return fine_score, evaluations

In [ ]:
patients = [
    '00000000034',
    '00000000074',
    '00000000058',
    '00000000039',
    '00000000081',
    '00000000042',
]

In [ ]:
for patient_id in patients:
    fine_score, evaluations = get_fine_score(patient_id)
    fig, ax = plt.subplots(figsize=(4,2)) 

    x = 1 + np.arange(len(evaluations))

    ax.plot(x, fine_score, color='tab:blue', marker='D')
    ax.bar(x, evaluations, color='tab:orange', edgecolor='white', linewidth=0.7)

    ax.set(
        xlim=(0, len(sessions) + 1), 
        xticks=np.arange(1, len(sessions) + 1),
        ylim=(0, max(7, max(fine_score) + 1)),
        xlabel='Sessions',
        ylabel='Scores',
        title='%s: HB scores %s ' % (patient_id, ', '.join(map(str, evaluations)))
    )

plt.show() 

__END__